In [20]:
import pandas as pd
import ast
import pyGM as gm
import numpy as np
import math
from random import random

In [2]:
#upload card db
cards = pd.read_csv('data/cards.csv')

In [3]:
# add converted mana cost as well as make all values not strings
cards['CMC'] = cards['manaCost'].apply(lambda x: ''.join(''.join(str(x).split('{')).split('}')))
for col in cards:
    try:
        cards[col] = cards[col].apply(lambda x: ast.literal_eval(x))
    except:
        pass
for index, row in cards.iterrows():
    X = row['CMC']
    list(X)
    x_ = 0
    for x in X:
        try:
            x_+=int(x)
        except:
            x_+=1
    row['CMC']=x_

In [4]:
# only keep modern legal cards and drop non useful cols
legal = []
for index, row in cards.iterrows():
    if 'modern' in row['legalities']:
        if row['legalities']['modern']=='Legal':
            legal.append(True)
        else:
            legal.append(False)
    else:
        legal.append(False)
cards = cards[legal]
cards.drop(columns=['legalities','Unnamed: 0','leadershipSkills','power','toughness','loyalty','uuid'],inplace=True)

In [5]:
cards

,colors,manaCost,name,subtypes,supertypes,text,type,types,hasNoDeckLimit,CMC
8,[R],{2}{R},Abandon Reason,[],[],"[discard, discard, cast, madness]",Instant,[Instant],NaN,3
10,[],{3},Abandoned Sarcophagus,[],[],"[cast, cycling, cycling, exile]",Artifact,[Artifact],NaN,3
11,[B],{3}{B},Abattoir Ghoul,[Zombie],[],[creature],Creature — Zombie,[Creature],NaN,4
13,[W],{3}{W},Abbey Griffin,[Griffin],[],[vigilance],Creature — Griffin,[Creature],NaN,4
15,[R],{1}{R},Abbot of Keral Keep,"[Human, Monk]",[],"[cast, creature, exile, play]",Creature — Human Monk,[Creature],NaN,2
...,...,...,...,...,...,...,...,...,...,...
21032,[B],{B},Zulaport Enforcer,"[Human, Warrior]",[],[counter],Creature — Human Warrior,[Creature],NaN,1
21034,"[B, U, W]",{1}{W}{U}{B},Zur the Enchanter,"[Human, Wizard]",[Legendary],"[search, enchantment, shuffle]",Legendary Creature — Human Wizard,[Creature],NaN,4
21035,[U],{3}{U},Zur's Weirding,[],[],"[play, reveal]",Enchantment,[Enchantment],NaN,4
21039,[R],{R},Zurgo Bellstriker,"[Orc, Warrior]",[Legendary],"[cast, dash]",Legendary Creature — Orc Warrior,[Creature],NaN,1


In [6]:
#upload decklists
decks = dict()
decks['modern2018'] = pd.read_csv('data/deckListmodernMetaDecks2018.csv')
decks['modern2019'] = pd.read_csv('data/deckListmodernMetaDecks2019.csv')
decks['modern2020'] = pd.read_csv('data/deckListmodernMetaDecks2020.csv')


In [7]:
#select only one archetype from decklists
decklists = dict()
decklists['modern2020'] = decks['modern2020'][decks['modern2020']['1']=='Eldrazi Aggro']['3']
decklists['modern2019'] = decks['modern2019'][decks['modern2019']['1']=='Eldrazi Aggro']['3']
decklists['modern2018'] = decks['modern2018'][decks['modern2018']['1']=='Eldrazi Aggro']['3']
for d in decklists:
    print(decklists[d],d)
    
decklists['modern2020'] = decklists['modern2020'].to_dict()[2]
decklists['modern2020'] = ast.literal_eval(decklists['modern2020'])
decklists['modern2019'] = decklists['modern2019'].to_dict()[4]
decklists['modern2019'] = ast.literal_eval(decklists['modern2019'])
decklists['modern2018'] = decklists['modern2018'].to_dict()[6]
decklists['modern2018'] = ast.literal_eval(decklists['modern2018'])


2    [{'LANDS': ['3 Blast Zone', '1 Cavern of Souls...
Name: 3, dtype: object modern2020
4    [{'LANDS': ['2 Blast Zone', '1 Cavern of Souls...
Name: 3, dtype: object modern2019
6    [{'LANDS': ['4 Blinkmoth Nexus', '4 Eldrazi Te...
Name: 3, dtype: object modern2018


In [8]:
x = len(decklists['modern2018'])
y = len(decklists['modern2019'])
z = len(decklists['modern2020'])
x = len(decklists['modern2018'][:int(x/4)])
y = len(decklists['modern2019'][:int(y/2)])
z = len(decklists['modern2020'])

In [9]:
x+y+z

230

In [10]:
#Generate dataframes with card frequencies for all years for main and side
# as well as correlation matrixes for cards
simMain = dict()
simSide = dict()
for D in decklists:
    for i,deck in enumerate(decklists[D]):
        for ty in deck:
            if ty=='SIDEBOARD':
                sim=simSide
            else:
                sim=simMain
            for card in deck[ty]:
                card = card.split(' ')
                freq = card[0]
                name = ' '.join(card[1:])
                if name not in sim:
                    sim[name] = list()
                    if D=='modern2020':
                        i_ = i+x+y
                    if D=='modern2019':
                        i_ = i+x
                    if D=='modern2018':
                        i_ = i
                    sim[name].append([i_,int(freq)])
                else:
                    isIn = False
                    for i_,freq_ in sim[name]:
                        if i_==i:
                            isIn=True
                            freq_+=int(freq)
                    if isIn==False:
                        sim[name].append([i,int(freq)])
                        
                        
sim=simSide
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(x+y+z):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1
corrSide=corr
dSide = d

sim=simMain
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(x+y+z):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1

corrMain = corr
dMain = d

In [11]:
#select multiple potential cards and impute those values
#https://towardsdatascience.com/preprocessing-regression-imputation-of-missing-continuous-values-f612179bafb4
import pandas as pd
import numpy as np
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
import random



In [12]:
# used to select card fewqeuencies for non lands
def F(x):
    if x < 0:
        return 0
    if x > 3:
        return 4
    else:
        return math.ceil(x)
# generate a 60 card decklist given a seed list of cards/freqs
def genDeck(seed):
    x_ = len(dMain)
    dMain.loc[x_] = None
    for i in range(len(dMain.columns)):
        dMain.loc[x_][i] = np.nan
    decksize = 0
    imputer = IterativeImputer(BayesianRidge())
    #impute current seed
    for s in seed:
        decksize+=seed[s]
    while decksize!=60:
        for card in dMain:
            if card in seed:
                dMain.loc[x_][card] = seed[card]
            else:
                dMain.loc[x_][card] = np.nan
        #impute deck givne current seed        
        impMain = pd.DataFrame(imputer.fit_transform(dMain.values),columns = dMain.columns)
        for card in impMain:
            #check that the card is legal
            if len(cards[cards['name']==card])==1:
                if card in ['Forest', 'Mountain', 'Island', 'Plains', 'Swamp', 'Wastes']:
                    impMain[card] = impMain[card].apply(lambda x: math.ceil(x))
                if card in ['Snow-Covered Island', 'Snow-Covered Forest', 'Snow-Covered Plain', 'Snow-Covered Mountain', 'Snow-Covered Swamp']:
                    impMain[card] = impMain[card].apply(lambda x: math.ceil(x))

                else:
                    impMain[card] = impMain[card].apply(F)
        #sort cards not in seed by their average correlatoin w the cards in the seed
        corrs = corrMain[list(seed.keys())].transpose().mean().sort_values(ascending=False).to_dict()
        to_inc = []
        i = 10
        #include top 10 or so cards into the seed (by frequency) at a time then reimpute till 60 cards in deck
        for c in corrs:
            newFreq = impMain.loc[x_][c]
            if c not in seed and newFreq >0:
                if decksize+newFreq<=60:
                    seed[c] = newFreq
                    decksize+=newFreq
                    i-=newFreq
                if i<=0:
                    break
    dMain.drop(index=x_,axis=1,inplace=True)
    return seed

In [13]:
# Decksize = 60
# #For a given partial decklist, check if the addition of a new card
# # satisfies the restrictions placed upon the deck by specifying a companion(s) for it
# # and return True if the addition of a new card would not violate the deck restrictions
# def meetsCRestrictions(Companions: list, Card: pd.Series(), Decklist: list) -> bool:
#     for i in Companions:
#         if i == "Lurrus of the Dream-Den" and ('Creature' in Card.types.tolist()[0] or 'Enchantment' in Card.types.tolist()[0] or 'Artifact' in Card.types.tolist()[0] or 'Planeswalker' in Card.types.tolist()[0]) and Card.CMC.tolist()[0] > 2:
#             return False
#         if i == "Keruga, the Macrosage" and 'Land' not in Card.types.tolist()[0] and Card.CMC.tolist()[0] < 3:
#             return False
#         if i == "Jegantha, the Wellspring" and Card.manaCost.tolist()[0].split('}') != len(set(Card.manaCost.tolist()[0].split('}'))):
#             return False
#         if i == "Obosh, the Preypiercer" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] %2 == 0:
#             return False
#         if i == "Gyruda, Doom of Depths" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] % 2 == 1:
#             return False
#         if i == "Umori, the Collector":
#             CardType = None
#             for ty in Decklist:
#                 for x in Decklist[ty]:
#                     name = ' '.join(x.split()[1:])
#                     x = cards[cards['name']==name]
#                     if CardType == None:
#                         CardType = x.types.tolist()[0]
#                     elif 'Land' in CardType:
#                         CardType = x.types.tolist()[0]
#                     elif len(x.types.tolist()[0])>1 and 'Land' not in x.types.tolist()[0]:
#                         return False
#                     else:
#                         if x.types.tolist()[0] != ["Land"] and x.types.tolist()[0] not in CardType:
#                             return False
#         if i == "Lutri, the Spellchaser":
#             for ty in Decklist:
#                 for x in Decklist[ty]:
#                     freq = int(x.split()[0])
#                     name  = ' '.join(x.split()[1:])
#                     if (freq >1 and ty!='LANDS') or name==Card.name.tolist()[0]:
#                         return False

#         if i == "Kaheera, the Orphaguard" and 'Creature' in Card.types.tolist()[0]  and ("Cat" not in Card.subtypes.tolist()[0]  or "Elemental" not in Card.subtypes.tolist()[0] or "Nightmare" not in Card.subtypes.tolist()[0] or "Dinosaur" not in Card.subtypes.tolist()[0] or "Beast" not in Card.subtypes.tolist()[0]):
#             return False
#         if i == "Yorion, Sky Nomad":
#             Decksize = 80
#     return True

In [58]:
def metriculate(deck):
    metric = dict()
    metric['INSTANTS and SORC.'] = []
    metric['LANDS'] = []
    metric['CREATURES'] = []
    metric['OTHER SPELLS']= []
    decksCMC = 0
    numLands = 0
    for card in deck:
        c = cards[cards['name']==card].to_dict()
        try:
            types = c['types']
            types = list(types.values())[0]
            cmc  = c['CMC']
            cmc   = list(cmc.values())[0]
            decksCMC+=cmc
        except:
            #skip currently illegal card
            pass
        if 'Instant' in types or 'Sorcery' in types:
            metric['INSTANTS and SORC.'].append([card,deck[card],cmc])
        elif 'Creature' in types:
            metric['CREATURES'].append([card,deck[card],cmc])
        elif 'Land' in types:
            numLands+=deck[card]
            metric['LANDS'].append([card,deck[card],0])
        else:
            metric['OTHER SPELLS'].append([card,deck[card],cmc])
    metric2 = dict()
    for ty in metric:
        metric2[ty] = [0,0]
    for ty in metric:
        for cardInfo in metric[ty]:
            metric2[ty][0] +=cardInfo[1]
            metric2[ty][1] +=cardInfo[2]

    numCards = -numLands

    for ty in metric2:
        try:
            metric2[ty][1]/= metric2[ty][0]
        except:
            metric2[ty][1]+=0
        numCards+=metric2[ty][0]
    for ty in metric2:
        try:
            metric2[ty][0]/=numCards
        except:
            metric2[ty][0]+=0
    try:
        metric2['DECKS CMC'] = [decksCMC/numCards]
    except:
        metric2['DECKS CMC'] = 0
    return metric2

def compare(metric,metric1):
    Amse = 0
    for ty in metric:
        for i, item in enumerate(metric[ty]):
            Amse += (abs(item**2-metric1[ty][i]**2))**.5
    return Amse

In [35]:
TrueDeck = dMain.loc[50].to_dict()
testDeck = dMain.loc[50].to_dict()
keys = list(TrueDeck.keys())
for card in keys:
    if random()>.25:
        testDeck[card]=0
    if testDeck[card]==0:
        testDeck.pop(card)
    if TrueDeck[card]==0:
        TrueDeck.pop(card)

In [36]:
TrueDeck

{'Blast Zone': 2.0,
 'Cavern of Souls': 1.0,
 'Eldrazi Temple': 4.0,
 'Scavenger Grounds': 1.0,
 'Tectonic Edge': 1.0,
 "Urza's Mine": 4.0,
 "Urza's Power Plant": 4.0,
 "Urza's Tower": 4.0,
 'Wastes': 1.0,
 'Matter Reshaper': 4.0,
 'Reality Smasher': 4.0,
 'Thought-Knot Seer': 4.0,
 'Walking Ballista': 3.0,
 'All Is Dust': 1.0,
 'Dismember': 2.0,
 'Chalice of the Void': 4.0,
 'Expedition Map': 4.0,
 'Karn, the Great Creator': 4.0,
 'Mind Stone': 1.0,
 'Ugin, the Ineffable': 1.0,
 'Spatial Contortion': 3.0,
 'Ghost Quarter': 1.0,
 'Warping Wail': 2.0,
 'Karn Liberated': 2.0,
 'Endbringer': 1.0,
 'Ugin, the Spirit Dragon': 1.0,
 'Sea Gate Wreckage': 1.0,
 'Ulamog, the Ceaseless Hunger': 1.0,
 'Forest': 1.0,
 'Once Upon a Time': 4.0,
 'Hangarback Walker': 1.0,
 'Sanctum of Ugin': 1.0}

In [46]:
testDeck

{'Eldrazi Temple': 4.0,
 'Scavenger Grounds': 1.0,
 'Chalice of the Void': 4.0,
 'Warping Wail': 2.0,
 'Sea Gate Wreckage': 1.0,
 'Ulamog, the Ceaseless Hunger': 1.0,
 'Once Upon a Time': 4.0,
 "Urza's Mine": 3.0,
 "Urza's Power Plant": 3.0,
 "Urza's Tower": 3.0,
 'Expedition Map': 3.0,
 'Walking Ballista': 2.0,
 'All Is Dust': 1.0,
 'Wastes': 2.0,
 'Karn, the Great Creator': 4.0,
 'Blast Zone': 2.0,
 'Ugin, the Ineffable': 2.0,
 'Matter Reshaper': 4.0,
 'Reality Smasher': 4.0,
 'Thought-Knot Seer': 4.0,
 'Tectonic Edge': 1.0,
 'Mind Stone': 1.0,
 'Endbringer': 1.0,
 'Hangarback Walker': 1.0,
 'Wurmcoil Engine': 1.0,
 'Karn Liberated': 1.0}

In [38]:
testDeck = genDeck(testDeck)

C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [23]:
avgMetric = dict()
avgMetric['INSTANTS and SORC.'] = [0,0]
avgMetric['CREATURES'] = [0,0]
avgMetric['LANDS'] = [0,0]
avgMetric['OTHER SPELLS'] = [0,0]
avgMetric['DECKS CMC'] = [0]

In [24]:
numDecks = 0
for index, row in dMain.iterrows():
    if sum(row.to_dict().values())!=0:
        metric = metriculate(row.to_dict())
        numDecks+=1
        for ty in metric:
            for i,item in enumerate(metric[ty]):
                avgMetric[ty][i]+=item
for ty in avgMetric:
    for i, item in enumerate(avgMetric[ty]):
        avgMetric[ty][i]/=numDecks

In [57]:
avgMetric

{'INSTANTS and SORC.': [0.15952458161040575, 2.4589416203193273],
 'CREATURES': [0.5319623696538155, 3.712542255400075],
 'LANDS': [0.6200021933380551, 0.0],
 'OTHER SPELLS': [0.3085130487357796, 4.854181127552218],
 'DECKS CMC': [3.0003701679949164]}

In [56]:
metricTrue= metriculate(TrueDeck)
metricTrue

{'INSTANTS and SORC.': [0.1702127659574468, 2.25],
 'LANDS': [0.5531914893617021, 0.0],
 'CREATURES': [0.3829787234042553, 1.2777777777777777],
 'OTHER SPELLS': [0.44680851063829785, 1.5714285714285714],
 'DECKS CMC': [2.3404255319148937]}

In [48]:
metricTest = metriculate(testDeck)
metricTest

{'INSTANTS and SORC.': [0.075, 3.0],
 'LANDS': [0.5, 0.0],
 'CREATURES': [0.45, 1.6111111111111112],
 'OTHER SPELLS': [0.475, 1.2105263157894737],
 'DECKS CMC': [2.175]}

In [42]:
#MSE of each value averaged

In [43]:
compare(metricTrue,metricTest)

5.618866495923664